In [38]:
!pip install pytorch-lightning -q

In [39]:
import pytorch_lightning as pl
import torch
from torchvision import datasets,models
import torchvision.transforms as tt
from torch.utils.data import Dataset,random_split,DataLoader
import torch.nn as nn
import torch.nn.functional as F

In [40]:
stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
train_tfms = tt.Compose([tt.Resize(256),
                         tt.CenterCrop(224),
                         tt.ToTensor(), 
                         tt.Normalize(*stats,inplace=True)])
valid_tfms = tt.Compose([tt.Resize([224,224]),tt.ToTensor(), tt.Normalize(*stats)])

In [41]:
ds = datasets.ImageFolder('food-101-subset/images/')

In [42]:
class CustomDataset(Dataset):
    def __init__(self,ds,transformer):
        self.ds = ds
        self.transform = transformer
    
    def __getitem__(self,idx):
        image,label = self.ds[idx]
        img = self.transform(image)
        return img,label
    
    def __len__(self):
        return len(ds)

In [43]:
train_len=0.8*len(ds)
val_len = len(ds) - train_len
int(train_len),int(val_len)

(16160, 4040)

In [44]:
train_ds,val_ds = random_split(dataset=ds,lengths=[int(train_len),int(val_len)],generator=torch.Generator().manual_seed(42))

In [45]:
t_ds = CustomDataset(train_ds.dataset,train_tfms)
v_ds = CustomDataset(val_ds.dataset,valid_tfms)

In [46]:
batch_size = 32
train_dl = DataLoader(t_ds, batch_size, shuffle=True, pin_memory=True)
valid_dl = DataLoader(v_ds, batch_size, pin_memory=True)

In [47]:
class Flatten(nn.Module):
    def forward(self,x):
        return torch.flatten(x,1)

In [73]:
class FoodImageClassifer(pl.LightningModule):
    def __init__(self,learning_rate,batch_size=32):
        super().__init__()
        mobilenet = models.mobilenet_v2(pretrained=True)
        self.body = mobilenet.features
        self.head = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(1280,101))
        self.learning_rate = learning_rate
        self.batch_size = batch_size
    
    def forward(self,x):
        x = self.body(x)
        x = nn.functional.adaptive_avg_pool2d(x, (1, 1))
        x = torch.flatten(x, 1) 
        return self.head(x)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        return loss
    
    def train_dataloader(self):
         return DataLoader(t_ds, self.batch_size, shuffle=True, pin_memory=True,num_workers=7)
    
    def val_dataloader(self):
        return DataLoader(v_ds, self.batch_size, pin_memory=True,num_workers=7)
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        val_loss = F.cross_entropy(y_hat, y)
        return val_loss
    
    def freeze(self):
        for name,param in self.body.named_parameters():
            param.requires_grad = True

In [74]:
model = FoodImageClassifer(1e-3)
trainer = pl.Trainer(gpus=1,auto_lr_find=True)
trainer.tune(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Running in fast_dev_run mode: will run a full train, val and test loop using 1 batch(es).
/opt/conda/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Skipping learning rate finder since fast_dev_run is enabled.
  warnings.warn(*args, **kwargs)


{'lr_find': None}

In [75]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params
------------------------------------
0 | body | Sequential | 2.2 M 
1 | head | Sequential | 129 K 
------------------------------------
2.4 M     Trainable params
0         Non-trainable params
2.4 M     Total params
9.413     Total estimated model params size (MB)


In [35]:
model = FoodImageClassifer()


{}